# RANK() und DENSE\_RANK()

- [Ranking Functions (Transact-SQL)](https:\learn.microsoft.com\en-us\sql\t-sql\functions\ranking-functions-transact-sql?view=sql-server-ver16)
- [RANK (Transact-SQL)](https:\learn.microsoft.com\en-us\sql\t-sql\functions\rank-transact-sql?view=sql-server-ver16)
- [DENSE\_RANK (Transact-SQL)](https:\learn.microsoft.com\en-us\sql\t-sql\functions\dense-rank-transact-sql?view=sql-server-ver16)

In [ ]:
-- Rang aller Länder nach Anzahl Kunden im Land
SELECT RANK() OVER (ORDER BY COUNT(KundenNr) DESC) AS Rang
    , DENSE_RANK() OVER (ORDER BY COUNT(KundenNr) DESC) AS [Rang ohne Lücken]
    , Land, COUNT(KundenNr) AS [Anzahl Kunden]
FROM Kunde
GROUP BY Land;

In [ ]:
-- Rang der Kunden nach Umsatz partitioniert nach Land
SELECT k.Land
    , RANK() OVER(
            PARTITION BY k.Land 
            ORDER BY SUM(d.Menge * d.Verkaufspreis) DESC
    ) AS [Rang im Land]
    , k.KundenNr, k.Firma
    , SUM(d.Menge * d.Verkaufspreis) AS [Umsatz]
FROM Kunde AS k 
JOIN Bestellung AS b ON k.KundenNr = b.KundenNr
JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
GROUP BY k.Land, k.KundenNr, k.Firma;

In [ ]:
-- Filtere die Top 3 Kunden nach Umsatz je Land
WITH Umsatzabfrage AS (
    SELECT k.Land
    , RANK() OVER(
            PARTITION BY k.Land 
            ORDER BY SUM(d.Menge * d.Verkaufspreis) DESC
    ) AS [Rang im Land]
    , k.KundenNr, k.Firma
    , SUM(d.Menge * d.Verkaufspreis) AS [Umsatz]
    FROM Kunde AS k 
    JOIN Bestellung AS b ON k.KundenNr = b.KundenNr
    JOIN BestellDetail AS d ON b.BestellungID = d.BestellungID
    GROUP BY k.Land, k.KundenNr, k.Firma
)
SELECT * 
FROM Umsatzabfrage
WHERE [Rang im Land] <= 3;

In [ ]:
-- Zeige die Top 3 Produkte nach Verkaufzahlen in jeder Kategorie
WITH u AS (
    SELECT k.Kategorie
        , p.Produkt    
        , RANK() OVER( 
            PARTITION BY k.Kategorie 
            ORDER BY SUM(d.Menge * d.Verkaufspreis) DESC
            ) AS [Rang des Produkts] 
        , SUM(d.Menge * d.Verkaufspreis) AS Umsatz
    FROM Produktkategorie AS k
    JOIN Produkt AS p ON k.KategorieID = p.KategorieID
    JOIN BestellDetail AS d ON p.ProduktID = d.ProduktID
    GROUP BY k.Kategorie, p.Produkt
)
SELECT * 
FROM u
WHERE [Rang des Produkts] <= 3;